This is the prototype of the Mean Field

There will be three main modules:

- 1) Agent

- 2) MeanField

- 3) BlackBoard

In [45]:

import numpy as np
import scipy as sp
import ode
from sliding_window import *

### Agent

In [46]:
class Agent:
    
    def __init__(self, mean_field, blackboard, state_indices, control_indices):
        '''
        state_indices (list of integers): This list tells which states pertain to this agent. e.g. [1,2] would 
        tell us that states 1 and 2 pertain to this agent.
        
        A word on notation:  The notation used for the methods of the agent is:  
            - If it is a partial derivative: <denominator>_rhs_H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g., 
            "qp_rhs_H_l_u" denotes the partial derivative with respect to q and p of the terms in the local Hamiltonian that contain control variables.
            - If it is a hamiltonian: H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g. "H_mf_nou" denotes the mean field hamiltonian
            with terms not containing u.
        '''
        self.state_indices = state_indices
        self.control_indices = control_indices
        self.mf = mean_field
        self.bb = blackboard

        # Inputs for numerical propagator
        # qp_vec is going to be [q_s, p_l, p_mf], so it will have dimension = 3*state_dim

        self.q_s_0 = np.array([0])
        self.p_l_0 = np.array([0])
        self.p_mf_0 = np.array([0])
        self.u_0 = np.array([0])
        self.qpu_vec = np.hstack([self.q_s_0, self.p_l_0, self.p_mf_0, self.u_0])
        self.state_dim = 1
        self.Gamma = 1 
        self.gamma = 1

        # Inputs for numerical integration
        self.integrateTol = 10**-3
        self.integrateMaxIter = 40

        # Inputs for sliding window
        self.t_0 = 0 
        self.T =  2
        self.K=1 

        self.t_terminal = 2
        self.n_s = 10

        self.validate_dimensions()

    def validate_dimensions(self):
        # TODO: move to parent class "SlidingWindow"
        assert len(self.state_indices) == self.state_dim, 'state dimensions are not consistent.  dimension of state indices is '+str(len(self.state_indices)) +' and state_dim is '+str(self.state_dim)
        assert len(self.control_indices) == len(self.u_0), 'control dimensions are not consistent.  dimension of control_indices is '+str(len(self.control_indices)) +' and len(u_0) is '+str(len(self.u_0))
        assert len(self.qpu_vec) == 3*self.state_dim + len(self.control_indices), ' control and state dimensions are not consistent with qpu_vec : length of qpu_vec is '+str(len(self.qpu_vec))+ ' and 3*self.state_dim + len(self.control_indices) is ' + str(3*self.state_dim + len(self.control_indices))
    
    '''
    TODO:  
    Add an assertion to check that the dimension of q_s_0, p_mf_0, and u_0:
        - the dimension of state_dim    
        - state_indices and control_indices set upon initiation of the Agent
        - 
    '''
    
    def L_l(self, q, q_dot, u):
        return
    
    def L_l_q_dot(self, t, q, q_dot, u, **kwargs):
        return
    
    def H_l_nou(self, q, p, lambda_l):
        return 

    def qp_rhs_H_l_nou(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/2
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot = np.zeros(np.shape(p))
        p_D_dot = np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_l_nou(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        returnurn -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_u(self, q, p, u):
        return 
    
    def qp_rhs_H_l_u(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot =  np.zeros(np.shape(p))
        p_D_dot =  np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_l_u(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_D(self, q_lD, p_lD):
        return np.array(q_lD).dot(p_lD) 
        
    def L_l_D(self, q_lD, p_lD):
        return np.array(q_lD).dot(p_lD)
        
    def L_l_D_q_Dot(self, q_lD, p_lD):
        return q_lD

    def qp_rhs(self, t, qp_vec, **kwargs):
        # u_s is constant (because of causality, remember?)
        u_s = kwargs['u_0']
        state_dim = kwargs['state_dim']
        q_s = qp_vec[:state_dim]
        p_l = qp_vec[state_dim:2*state_dim]
        p_mf = qp_vec[2*state_dim:]

        qp_rhs_H_mf = self.mf.qp_rhs_H_mf(q_s, p_mf, p_l, u_s)
#         import pdb; pdb.set_trace()        
        p_rhs_H_mf = qp_rhs_H_mf[:state_dim]
        q_rhs_H_mf = qp_rhs_H_mf[state_dim:]
        
        qp_rhs_H_l = self.qp_rhs_H_l(q_s, p_l, u_s)
        q_rhs_H_l = qp_rhs_H_l[:state_dim]
        p_rhs_H_l = qp_rhs_H_l[state_dim:]

        q_s_dot = self.gamma*q_rhs_H_mf + (1-self.gamma)*q_rhs_H_l
        p_mf_dot = p_rhs_H_mf
        p_l_dot = -1*p_rhs_H_l
        
        return np.concatenate([q_s_dot, p_l_dot, p_mf_dot])
    
    def qp_rhs_H_l(self, q_s, p_l, u_s):
        q_rhs_H_l = self.q_rhs_H_l(q_s, p_l, u_s)
        p_rhs_H_l = self.p_rhs_H_l(q_s, p_l, u_s)
        return np.concatenate([q_rhs_H_l, p_rhs_H_l])
    
    def p_rhs_H_l(self, q_s, p_l, u_s):
        q_s_dot =  q_s+p_l+u_s 
        return q_s_dot
    
    def q_rhs_H_l(self, q_s, p_l, u_s):
        p_l_dot =  q_s + p_l + u_s
        return p_l_dot

    def u_rhs(self, t, u_vec, **kwargs):
        return -1*self.Gamma*np.zeros(np.shape(u_vec)) 

    ## Mean Field methods 
    def H_MF_nou(self, q_s, p_mf, p_l, u_s):
        return 1

    def H_MF_u(self, q_s, p_mf, p_l, u_s):
        return 1
        
    def qp_rhs_H_mf(self, q_s, p_mf, p_l, u_s):
        # remember that we want to propagate as much as possible together in the same rhs function for numerical purposes
        # remember that q_rhs here is w.r.t p_mf but p_rhs here is w.r.t q_s
        q_H_mf_dot = self.p_rhs_H_mf(q_s, p_mf, p_l, u_s)
        p_H_mf_dot = self.q_rhs_H_mf(q_s, p_mf, u_s)
        return np.concatenate([q_H_mf_dot, p_H_mf_dot])
    
    def q_rhs_H_mf(self, q_s, p_mf, u_s):
        p_H_mf_dot = self.q_rhs_H_mf_nou(q_s, p_mf) + sum(self.q_rhs_H_mf_u(q_s, p_mf)*u for i,j in zip())
        return p_H_mf_dot
        
    def q_rhs_H_mf_u(self, q_s, p_mf):
        # there should be one of these for each control variable
        p_H_mf_u_dot_1 = q_s + p_mf # or something
        return np.concatenate([p_H_mf_u_dot_1]) #, p_H_mf_u_dot_2])
    
    def p_rhs_H_mf(self, q_s, p_mf, p_l, u_s):
        return self.p_rhs_H_mf_nou(q_s, p_mf) + sum([self.p_rhs_H_mf_u(q_s, p_mf)*u for i,j in zip()])

    def p_rhs_H_mf_nou(self, q_s, p_mf):
        return q_s+p_mf # or something

    def q_rhs_H_mf_nou(self, q_s, p_mf):
        p_H_mf_nou_dot_1 = q_s + p_mf
        return np.concatenate([p_H_mf_nou_dot_1]) #, p_H_mf_nou_dot_2])

    def p_rhs_H_mf_u(self, q_s, p_mf):
        return 


In [47]:
class Agent2:
    
    def __init__(self, mean_field, blackboard, state_indices, control_indices):
        '''
        state_indices (list of integers): This list tells which states pertain to this agent. e.g. [1,2] would 
        tell us that states 1 and 2 pertain to this agent.
        
        A word on notation:  The notation used for the methods of the agent is:  
            - If it is a partial derivative: <denominator>_rhs_H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g., 
            "qp_rhs_H_l_u" denotes the partial derivative with respect to q and p of the terms in the local Hamiltonian that contain control variables.
            - If it is a hamiltonian: H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g. "H_mf_nou" denotes the mean field hamiltonian
            with terms not containing u.
        '''
        self.state_indices = state_indices
        self.control_indices = control_indices
        self.mf = mean_field
        self.bb = blackboard

        # Inputs for numerical propagator

        self.q_s_0 = np.array([0,2])
        self.p_l_0 = np.array([0,3])
        self.p_mf_0 = np.array([0,1])
        self.u_0 = np.array([0])
        self.qpu_vec = np.hstack([self.q_s_0, self.p_l_0, self.p_mf_0, self.u_0])
        self.state_dim = 2
        self.Gamma = 1 
        self.gamma = 1

        # Inputs for numerical integration
        self.integrateTol = 10**-3
        self.integrateMaxIter = 40

        # Inputs for sliding window
        self.t_0 = 0 
        self.T =  2
        self.K=1 

        self.t_terminal = 2
        self.n_s = 10

        self.validate_dimensions()

    def validate_dimensions(self):
        # TODO: move to parent class "SlidingWindow"
        assert len(self.state_indices) == self.state_dim, 'state dimensions are not consistent.  dimension of state indices is '+str(len(self.state_indices)) +' and state_dim is '+str(self.state_dim)
        assert len(self.control_indices) == len(self.u_0), 'control dimensions are not consistent.  dimension of control_indices is '+str(len(self.control_indices)) +' and len(u_0) is '+str(len(self.u_0))
        assert len(self.qpu_vec) == 3*self.state_dim + len(self.control_indices), ' control and state dimensions are not consistent with qpu_vec : length of qpu_vec is '+str(len(self.qpu_vec))+ ' and 3*self.state_dim + len(self.control_indices) is ' + str(3*self.state_dim + len(self.control_indices))
    
    '''
    TODO:  
    Add an assertion to check that the dimension of q_s_0, p_mf_0, and u_0:
        - the dimension of state_dim    
        - state_indices and control_indices set upon initiation of the Agent
        - 
    '''
    
    def L_l(self, q, q_dot, u):
        return
    
    def L_l_q_dot(self, t, q, q_dot, u, **kwargs):
        return
    
    def H_l_nou(self, q, p, lambda_l):
        return 

    def qp_rhs_H_l_nou(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/2
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot = np.zeros(np.shape(p))
        p_D_dot = np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_l_nou(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        returnurn -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_u(self, q, p, u):
        return 
    
    def qp_rhs_H_l_u(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot =  np.zeros(np.shape(p))
        p_D_dot =  np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_l_u(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_D():
        pass
        
    def L_l_D():
        pass
        
    def L_l_D_q_Dot():
        pass

    def qp_rhs(self, t, qp_vec, **kwargs):
        # u_s is constant (because of causality, remember?)
        u_s = kwargs['u_0']
        state_dim = kwargs['state_dim']
        q_s = qp_vec[:state_dim]
        p_l = qp_vec[state_dim:2*state_dim]
        p_mf = qp_vec[2*state_dim:]

        qp_rhs_H_mf = self.mf.qp_rhs_H_mf(q_s, p_mf, p_l, u_s)
        
        p_rhs_H_mf = qp_rhs_H_mf[:state_dim]
        q_rhs_H_mf = qp_rhs_H_mf[state_dim:]
        
        qp_rhs_H_l = self.qp_rhs_H_l(q_s, p_l, u_s)
        q_rhs_H_l = qp_rhs_H_l[:state_dim]
        p_rhs_H_l = qp_rhs_H_l[state_dim:]
        
        q_s_dot = self.gamma*q_rhs_H_mf + (1-self.gamma)*q_rhs_H_l
        p_mf_dot = p_rhs_H_mf
        p_l_dot = -1*p_rhs_H_l
        
        return np.concatenate([q_s_dot, p_l_dot, p_mf_dot])
    
    def qp_rhs_H_l(self, q_s, p_l, u_s):
        q_rhs_H_l = self.q_rhs_H_l(q_s, p_l, u_s)
        p_rhs_H_l = self.p_rhs_H_l(q_s, p_l, u_s)
        return np.concatenate([q_rhs_H_l, p_rhs_H_l])
    
    def p_rhs_H_l(self, q_s, p_l, u_s):
        q_s_dot =  q_s+p_l+u_s 
        return q_s_dot
    
    def q_rhs_H_l(self, q_s, p_l, u_s):
        p_l_dot =  q_s + p_l + u_s
        return p_l_dot
    
    def u_rhs(self, t, u_vec, **kwargs):
        return -1*self.Gamma*np.zeros(np.shape(u_vec)) 

    ## Mean Field methods 
    def H_MF_nou(self, q, p):
        pass

    def H_MF_u(self, q, p):
        pass
        
    def qp_rhs_H_mf(self, q_s, p_mf, p_l, u_s):
        # remember that we want to propagate as much as possible together in the same rhs function for numerical purposes
        # remember that q_rhs here is w.r.t p_mf but p_rhs here is w.r.t q_s
        q_H_mf_dot = self.p_rhs_H_mf(q_s, p_mf, p_l, u_s)
        p_H_mf_dot = self.q_rhs_H_mf(q_s, p_mf, u_s)
        return np.concatenate([q_H_mf_dot, p_H_mf_dot])
    
    def q_rhs_H_mf(self, q_s, p_mf, u_s):
        p_H_mf_dot = self.q_rhs_H_mf_nou(q_s, p_mf) + sum(self.q_rhs_H_mf_u(q_s, p_mf)*u for i,j in zip())
        return p_H_mf_dot
        
    def q_rhs_H_mf_u(self, q_s, p_mf):
        # there should be one of these for each control variable
        p_H_mf_u_dot_1 = q_s + p_mf # or something
#         p_H_mf_u_dot_2 = q_s[1] + p_mf[1] # or something
        return np.concatenate([p_H_mf_u_dot_1]) #, p_H_mf_u_dot_2])
    
    def p_rhs_H_mf(self, q_s, p_mf, p_l, u_s):
        return self.p_rhs_H_mf_nou(q_s, p_mf) + sum([self.p_rhs_H_mf_u(q_s, p_mf)*u for i,j in zip()])

    def p_rhs_H_mf_nou(self, q_s, p_mf):
        return q_s+p_mf # or something

    def q_rhs_H_mf_nou(self, q_s, p_mf):
        p_H_mf_nou_dot_1 = q_s + p_mf
        return np.concatenate([p_H_mf_nou_dot_1]) #, p_H_mf_nou_dot_2])

    def p_rhs_H_mf_u():
        pass


## MeanField

In [48]:
class MeanField:
    
    def __init__(self):
        pass
    
#     def H_MF_nou(self, q, p):
#         pass

#     def H_MF_u(self, q, p):
#         pass
        
#     def qp_rhs_H_mf(self, q_s, p_mf, p_l, u_s):
#         # remember that we want to propagate as much as possible together in the same rhs function for numerical purposes
#         # remember that q_rhs here is w.r.t p_mf but p_rhs here is w.r.t q_s
#         q_H_mf_dot = self.p_rhs_H_mf(q_s, p_mf, p_l, u_s)
#         p_H_mf_dot = self.q_rhs_H_mf(q_s, p_mf, u_s)
#         return np.concatenate([q_H_mf_dot, p_H_mf_dot])
    
#     def q_rhs_H_mf(self, q_s, p_mf, u_s):
#         p_H_mf_dot = self.q_rhs_H_mf_nou(q_s, p_mf) + sum(self.q_rhs_H_mf_u(q_s, p_mf)*u for i,j in zip())
#         return p_H_mf_dot
        
#     def q_rhs_H_mf_u(self, q_s, p_mf):
#         # there should be one of these for each control variable
#         p_H_mf_u_dot_1 = q_s + p_mf # or something
# #         p_H_mf_u_dot_2 = q_s[1] + p_mf[1] # or something
#         return np.concatenate([p_H_mf_u_dot_1]) #, p_H_mf_u_dot_2])
    
#     def p_rhs_H_mf(self, q_s, p_mf, p_l, u_s):
#         return self.p_rhs_H_mf_nou(q_s, p_mf) + sum([self.p_rhs_H_mf_u(q_s, p_mf)*u for i,j in zip()])

#     def p_rhs_H_mf_nou(self, q_s, p_mf):
#         return q_s+p_mf # or something

#     def q_rhs_H_mf_nou(self, q_s, p_mf):
#         p_H_mf_nou_dot_1 = q_s + p_mf
#         return np.concatenate([p_H_mf_nou_dot_1]) #, p_H_mf_nou_dot_2])

#     def p_rhs_H_mf_u():
#         pass
          

In [49]:
a=[3,5,2]
b=[2]
g=[a,b]
set([x for g_i in g for x in g_i]) - set([3,3,3,2,4])
print 'only 5 should be the resulting set'

only 5 should be the resulting set


In [50]:
class Synchronizer:
    
    def __init__(self, agents, blackboard):
        self.agents = agents  # list of all agents.  list with elements of class Agent
        self.bb = blackboard  # instance of class blackboard  
    
    def synchronize():
        # run synchronization
        for agent in self.agents:
            '''     
            1) run synchronized propagation - I think we only need one Agent instead of SlidingWindow now

            For each of the above 2 steps:
                - create sliding window instance
                - call "propagate_dynamics" on the sliding window instance

            get quenched values from blackboard
            '''
            # determine dimensions of p_l and p_MF, and then create
            # dimensions of p_l are determined by number of states for this agent
            # dimensions of p_MF
            state_dim_l = len(agent.state_indices)
            # set difference between all states in all agents, and states in current agent
            state_dim_mf = set([agent_i.state_indices for agent_i in self.agents for agent_i.state_indices in agent_i]) - set(agent.state_indices)
            # run propagation with keyword arguments state_dim_l, state_dim_mf

            qp_vec = self.bb.construct_q_p(agent) # quench the necessary values to be quenched
            qp_vec = qp_rhs_H_s_nou(qp_vec)

### Blackboard

In [51]:
class Blackboard:
    
    def __init__(self):
        '''
#         state_agent_mapping is dictionary with keys: object of class Agent, 
#               values: list of integers state_indices state_indices
        q_p_u_dict is a dictionary which maps 'q', 'p', 'u', to a dictionary of index-value pairs: local values for q, p, and u for this agent.  
                        blackboard holds all of the most recent local values, e.g.
                        {'q_s': {'1':3, '2':0}, 'p_mf': {'1':3, '2': 2}, 'u': {'1': 0}}
                        It doesn't care which agent updated them most recently.  It only needs to know which values to update.
        q_p_u_dict initially will be filled
        '''
        self.q_p_u_dict={'q_s':{}, 'p_l':{}, 'p_mf':{}, 'u_s':{}}
        self.agents=[]
    
    def construct_p_q(self, agent):
        '''
        Inputs:
            agent (instance of class Agent): This is the agent for which we are constructing p and q.
        
        Outputs:
            q_p_u_dict (dictionary): This dictionary holds all of the q, p, and u values for ALL of the agents
        '''
        assert agent in self.agents, 'this agent\'s values have not been added to the blackboard.  please call "self.update_q_p_u_dict(<agent>)" before calling this method'
#         q_p_u_dict = {'q_s':{}, 'p_l':{}, 'p_mf':{}, 'u_s':{}}
#         # find which states pertain to this agent
#         # we cannot just return self.q_p_u_dict because the dimensions are different for different agents
#         # for the states that don't, get the values from q_p_u_dict and put them into qp_vec
#         print agent.state_indices
#         for state_ix in agent.state_indices:
#             q_p_u_dict['q_s'][str(state_ix)] = self.q_p_u_dict['q_s'][str(state_ix)]
#             q_p_u_dict['p_l'][str(state_ix)] = self.q_p_u_dict['p_l'][str(state_ix)]
#             q_p_u_dict['p_mf'][str(state_ix)] = self.q_p_u_dict['p_mf'][str(state_ix)]

#         for control_ix in agent.control_indices:
#             q_p_u_dict['u_s'][str(control_ix)] = self.q_p_u_dict['u_s'][str(control_ix)]

#         # for state indices which are not part of agent.state_indices, fill in from blackboard q_p_u_dict
#         assert len(set([str(x) for x in agent.control_indices]) - set(self.q_p_u_dict['u_s'].keys())) == 0, ' there should not be any controls in agent that are not in blackboard '
#         assert len(set([str(x) for x in agent.state_indices]) - set(self.q_p_u_dict['q_s'].keys())) == 0, ' there should not be any states in agent that are not in blackboard '

#         # blackboard - agent
#         # TODO: is "set(self.q_p_u_dict['q_s'].keys())" the best way to find all of the state indices in existence? or should we just iterate over [agent.state_indices for agent in self.agents]
#         quenched_state_indices = set([str(x) for x in agent.state_indices]) - set(self.q_p_u_dict['q_s'].keys())
#         print quenched_state_indices
#         # fill in with values from blackboard
#         for state_ix in quenched_state_indices:
#             q_p_u_dict['q_s'][str(state_ix)] = self.q_p_u_dict['q_s'][str(state_ix)]
#             q_p_u_dict['p_l'][str(state_ix)] = self.q_p_u_dict['p_l'][str(state_ix)]
#             q_p_u_dict['p_mf'][str(state_ix)] = self.q_p_u_dict['p_mf'][str(state_ix)]
            
        return self.q_p_u_dict
    
    def update_q_p_u_dict(self, agent):
        '''  This method should be called after local propagation of each agent
        Inputs:
            agent (instance of class Agent): this is the agent whose values we are updating
        Outputs:
            No outputs.  This method just updates the attributes of the blackboard,
            just update the dictionary, agent_q_p_u_dict.
        '''
        # determine which states pertain to this agent and replace the old values with new

        for state_ix in agent.state_indices:
            self.q_p_u_dict['q_s'][str(state_ix)] = agent.qpu_vec[:agent.state_dim][state_ix-1]
            self.q_p_u_dict['p_l'][str(state_ix)] = agent.qpu_vec[agent.state_dim:2*agent.state_dim][state_ix-1]
            self.q_p_u_dict['p_mf'][str(state_ix)] = agent.qpu_vec[2*agent.state_dim:3*agent.state_dim][state_ix-1]
            
        for control_ix in agent.control_indices:
            self.q_p_u_dict['u_s'][str(control_ix)] = agent.qpu_vec[3*agent.state_dim:][control_ix-1]
            
        if agent not in self.agents:
            self.agents.append(agent)


## Small test for two agents

- Add agents to blackboard and meanfield
- Run synchronizer to visit the agents

In [62]:
bb = Blackboard()
mean_field = MeanField()

myAgent=Agent(mean_field, bb, state_indices=[1], control_indices=[1])
myAgent2=Agent2(mean_field, bb, state_indices=[1,2], control_indices=[1])

In [63]:
bb.update_q_p_u_dict(myAgent)

In [64]:
bb.q_p_u_dict

{'p_l': {'1': 0}, 'p_mf': {'1': 0}, 'q_s': {'1': 0}, 'u_s': {'1': 0}}

In [55]:
q_p_u_dict = bb.construct_p_q(myAgent)

In [56]:
q_p_u_dict

{'p_l': {'1': 0}, 'p_mf': {'1': 0}, 'q_s': {'1': 0}, 'u_s': {'1': 0}}

### Try a very small test

Steps to test qp_rhs and u_rhs:

- Give some arbitrary initial conditions
- create Agent
- create Blackboard
- create MeanField
- connect those three things above.
- somehow create a sliding window instance, or at least wrangle the Agent into a Sliding Window instance

In [59]:
# forgot to pass in 'u_s' as a keyword argument
qpu_vec, q_ss_bar, p_ls_bar, p_mfs_bar, u_bar, q_ss, p_ls, p_mfs, us = propagate_dynamics(myAgent)
# forgot to pass in 'u_s' as a keyword argument
qpu_vec2, q_ss_bar2, p_ls_bar2, p_mfs_bar2, u_bar2, q_ss2, p_ls2, p_mfs2, us2 = propagate_dynamics(myAgent2)

Testing plan:
    - Test this on a few nonzero inputs
    - Test this on multidimensional state
    - Test this on multidimensional control
    - Get this working for multiple agents
    
   

In [60]:
qpu_vec, q_ss_bar, p_ls_bar, p_mfs_bar, u_bar, q_ss, p_ls, p_mfs, us

(array([0., 0., 0., 0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 [array([0.])],
 [array([0.])],
 [array([0.])],
 [array([0.])])

In [61]:
qpu_vec2, q_ss_bar2, p_ls_bar2, p_mfs_bar2, u_bar2, q_ss2, p_ls2, p_mfs2, us2

(array([  0.        ,  82.30417415,   0.        , -27.22676579,
          0.        ,  81.30417415,   0.        ]),
 array([ 0.        , 11.57728582]),
 array([ 0.        , -2.72097727]),
 array([ 0.        , 10.57728582]),
 array([0.]),
 [array([ 0.        , 11.57728582])],
 [array([ 0.        , -2.72097727])],
 [array([ 0.        , 10.57728582])],
 [array([0.])])